In [5]:
import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
import pickle
import carla
import random
import time
import open3d as o3d

from .constants import *
from .utils import *

# connect to the sim 
client = carla.Client('localhost', 2000)

time.sleep(5)
client.set_timeout(25)
#client.load_world('Town03')


# get world and spawn points
world = client.get_world()

# ensure sync mode on 
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.1
settings.no_rendering_mode = False
world.apply_settings(settings)

spawn_points = world.get_map().get_spawn_points()

#clean up any existing cars
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

#look for a blueprint of Tesla m3 car
vehicle_bp = world.get_blueprint_library().filter('*model3*')[0]

def obstacle_callback(event):
    print(f"Obstacle detected at distance: {event.distance} meters, with other_actor ID: {event.other_actor.id}")
    global obstacle_detected
    obstacle_detected = True

pcd_data1 = []

def lidar_callback(point_cloud, point_list):
    """Prepares a point cloud with intensity
    colors ready to be consumed by Open3D"""
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))

    points = data[:, :-1]

    points[:, :1] = -points[:, :1]

    point_list.points = o3d.utility.Vector3dVector(points)
    # point_list.colors = o3d.utility.Vector3dVector(int_color)
    pcd_data1.append(points)



data_path = r"E:\Download\Hung\15-11-2024\data"
img_counter = 0
quit = False

while img_counter < 10000:
    cleanup(world)
    start_point = random.choice(spawn_points)
    vehicle = world.spawn_actor(vehicle_bp, start_point)
    # setting semantic camera
    camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
    camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.13 on Windows
    camera_bp.set_attribute('image_size_y', '480')
    camera_bp.set_attribute('fov', '90')
    camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
    camera_sem = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
    image_w = 640
    image_h = 480

    camera_data = {'sem_image': np.zeros((image_h,image_w,4)),
                    'rgb_image': np.zeros((image_h,image_w,4))}

    camera_sem.listen(lambda image: sem_callback(image,camera_data))
    collision_data ={'collision': False}
    collision_bp = world.get_blueprint_library().find('sensor.other.collision')
    collision_sensor = world.spawn_actor(collision_bp,carla.Transform(), attach_to = vehicle)
    collision_sensor.listen(lambda event: collision_callback(event,collision_data))  
    obstacle_detected = False
    obstacle_bp = world.get_blueprint_library().find('sensor.other.obstacle')
    obstacle_bp.set_attribute('distance', '0.8')  # Set detection distance (e.g., 10 meters)
    obstacle_bp.set_attribute('hit_radius', '0.08')  # Set detection radius (e.g., 1 meter)
    obstacle_bp.set_attribute('only_dynamics', 'False')  # Detect both static and dynamic objects
    obstacle_bp.set_attribute('debug_linetrace', 'False')  # Optional: enable visualization for debugging

    # Attach the obstacle detector to the vehicle
    obstacle_transform = carla.Transform(carla.Location(x=2.5, z=1.0))  # Position ahead of the vehicle
    obstacle_sensor = world.spawn_actor(obstacle_bp, obstacle_transform, attach_to=vehicle)

    # Set up the listener for the obstacle sensor
    obstacle_sensor.listen(lambda event: obstacle_callback(event))  
    
    blueprint_library = world.get_blueprint_library()
    lidar_bp = generate_lidar_bp(blueprint_library)
    lidar_transform = carla.Transform(carla.Location(x=0, z=2))
    lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=vehicle)
    point_list = o3d.geometry.PointCloud()
    lidar.listen(lambda data: lidar_callback(data, point_list))

    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=False)
    vis.get_render_option().background_color = [0.05, 0.05, 0.05]
    vis.get_render_option().point_size = 1
    vis.get_render_option().show_coordinate_frame = True

    prev_position = vehicle.get_transform()
    # getting a random route for the car
    route = select_random_route(start_point,spawn_points)
    curr_wp = 5 #we will be tracking waypoints in the route and switch to next one when we get close to current one
    predicted_angle = 0
    PREFERRED_SPEED = 40 # setting speed at start of new route
    frame = 0
    # print(start_point, route)
    generate_traffic(world=world, client=client)
    while curr_wp<len(route)-1 and not collision_data['collision']:
        frame += 1
        # Carla Tick
        world.tick()
        
        if frame == 2:
            vis.add_geometry(point_list)
        vis.update_geometry(point_list)

        vis.poll_events()
        vis.update_renderer()
        
        if curr_wp >=len(route)-5: # within 10 points of end, the route is done
            cleanup(world)
            break
        while curr_wp<len(route)-2 and vehicle.get_transform().location.distance(route[curr_wp][0].transform.location)<5:
            curr_wp +=1 #move to next wp if we are too close
        curr_wp, predicted_angle = get_proper_angle(vehicle,curr_wp,route)
        gen_dir_angle = get_distant_angle(vehicle,curr_wp,route,30)
        
        v = vehicle.get_velocity()
        speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
        estimated_throttle = maintain_speed(speed)

        steer_input = predicted_angle
        current_position = vehicle.get_transform()
        sem_im = camera_data['sem_image']
        
        if obstacle_detected:
            print("Obstacle Detected")
            vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=1.0))
            v = vehicle.get_velocity()
            speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
            # if current_position.location.distance(prev_position.location)>5:
            prev_position = current_position
            img_counter += 1
            time_grab = time.time_ns()
            cv2.imwrite(f'{data_path}\\seggt\\%06d_%s_%s_%s_%s.png' % (time_grab, 1, 0, round(predicted_angle,0), gen_dir_angle), sem_im)
            convert_lidar_to_pgm_polar(pcd_data1[-1], output_filename=f'{data_path}\\pgm\\{time_grab}.pgm')
            print(img_counter)
            obstacle_detected = False
        else:
            steer_input = predicted_angle
            if predicted_angle<-MAX_STEER_DEGREES:
                steer_input = -MAX_STEER_DEGREES
            elif predicted_angle>MAX_STEER_DEGREES:
                steer_input = MAX_STEER_DEGREES
                
            vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input / STEERING_CONVERSION))
            v = vehicle.get_velocity()
            speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
            
            if current_position.location.distance(prev_position.location)>5:
                prev_position = current_position
                img_counter += 1
                time_grab = time.time_ns()
                cv2.imwrite(f'{data_path}\\seggt\\%06d_%s_%s_%s_%s.png' % (time_grab, 0, speed, round(predicted_angle,0), gen_dir_angle), sem_im)
                convert_lidar_to_pgm_polar(pcd_data1[-1], output_filename=f'{data_path}\\pgm\\{time_grab}.pgm')
                print(img_counter)

    if quit:
        break
cleanup(world)


Spawned 60 vehicles and 10 pedestrians.
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
Obstacle detected at distance: 0.4225916564464569 meters, with other_actor ID: 5028
Obstacle Detected
38
Obstacle detected at distance: 0.05757096782326698 meters, with other_actor ID: 5028
Obstacle Detected
39
Spawned 60 vehicles and 9 pedestrians.
40
41
42
43
44
45
46
47
48
49
50
Obstacle detected at distance: 0.5208470225334167 meters, with other_actor ID: 5089
Obstacle Detected
51
Obstacle detected at distance: 0.13424617052078247 meters, with other_actor ID: 5089
Obstacle Detected
52
Spawned 60 vehicles and 10 pedestrians.
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
Obstacle detected at distance: 0.0 meters, with other_actor ID: 5163
Obstacle Detected
114
Spawned 60 vehicles and 10 pedestria

KeyboardInterrupt: 